## 2.7: Refining and Presenting a Dashboard

This notebook walks through the steps to load data, process key metrics, and visualize results for the CitiBike dashboard.

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
from pathlib import Path
import os
from plotly.subplots import make_subplots
import streamlit as st
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from datetime import datetime as dt
from keplergl import KeplerGl
from PIL import Image
from numerize import numerize

C:\Users\henry\anaconda3\envs\viz-kep\lib\site-packages\keplergl\keplergl.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_string


In [2]:
# Auto-detect root 
twd = Path.cwd()
if twd.name.lower() in {'scripts','data'}:
    ROOT = twd.parent
else:
    ROOT = twd

DATA_DIR = ROOT / 'Data'
VIS_DIR  = ROOT / 'Visualizations'
os.makedirs(VIS_DIR, exist_ok=True)

DATA_DIR = Path(DATA_DIR) if not isinstance(DATA_DIR, Path) else DATA_DIR
OUT = DATA_DIR / "merged_trips_weather_2022_sample.csv"


## Load Data

In [7]:
# File paths
trips_fp   = DATA_DIR / 'citibike_weather_2022.csv'
weather_fp = DATA_DIR / 'laguardia_2022_weather.csv'

# Read CSVs
trips   = pd.read_csv(trips_fp, parse_dates=['started_at','ended_at','date'], low_memory=False)
weather = pd.read_csv(weather_fp, parse_dates=['date'])
trips_fp = Path(trips_fp)

print('Trips shape:', trips.shape)
print('Weather shape:', weather.shape)

NameError: name 'DATA_DIR' is not defined

Generate Streamlit App Script

In [10]:
script = f"""import streamlit as st
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load data
top20  = pd.read_csv(r"{top20_fp}")
merged = pd.read_csv(r"{merged_fp}")

st.set_page_config(page_title='CitiBike Dashboard', layout='wide')
st.title('CitiBike Strategy Dashboard')

# Bar chart
fig1 = go.Figure(go.Bar(x=top20['{tation_col}'], y=top20['value'], marker={{'color':top20['value'],'colorscale':'Blues_r'}}))
fig1.update_layout(title='Top 20 Stations', xaxis_title='Station', yaxis_title='Trips')
st.plotly_chart(fig1, use_container_width=True)

# Dual-axis chart
fig2 = make_subplots(specs=[[{{'secondary_y':True}}]])
fig2.add_trace(go.Scatter(x=merged['date'], y=merged['bike_rides_daily'], name='Trips'), secondary_y=False)
fig2.add_trace(go.Scatter(x=merged['date'], y=merged['TMAX'], name='TMAX'), secondary_y=True)
fig2.update_layout(title='Daily Trips vs TMAX')
st.plotly_chart(fig2, use_container_width=True)

# Add Kepler.gl Map
map_html = 'kepler_popular_trips_map.html'
with open(map_html,'r') as f:
    html_data = f.read()
st.header('Aggregated Bike Trips Map')
st.components.v1.html(html_data, height=800)
"""
with open('st_dashboard.py','w') as f:
    f.write(script)
print('Created st_dashboard.py with charts and map embed')

Created st_dashboard.py with charts and map embed


In [9]:
# Load (parse 'date' only if present)
first = pd.read_csv(trips_fp, nrows=1)
parse_dates = ["date"] if "date" in first.columns else None
df_full = pd.read_csv(trips_fp, parse_dates=parse_dates, low_memory=False)

# Keep only columns the dashboard uses (reduces size)
keep = [c for c in ["date","trip_count","bike_rides_daily","avgTemp",
                    "rideable_type","start_lat","start_lng","start_station_name"]
        if c in df_full.columns]
use = df_full[keep].copy() if keep else df_full.copy()

# Reproducible sampling (seed = 32)
np.random.seed(32)
frac = 0.08
while True:
    sample = use.sample(frac=frac, random_state=32)
    sample.to_csv(OUT, index=False)
    size_mb = OUT.stat().st_size / (1024*1024)
    print(f"frac={frac:.4f} -> {size_mb:.2f} MB ({len(sample):,} rows)")
    if size_mb < 25.0 or frac <= 0.01:
        break
    frac *= 0.85

print(f"SAVED: {OUT}  ({size_mb:.2f} MB)")


frac=0.0800 -> 4.59 MB (62,959 rows)
SAVED: C:\Users\henry\OneDrive\citibike-dashboard-2022\Data\merged_trips_weather_2022_sample.csv  (4.59 MB)


In [2]:
# Load reduced sample (<25 MB)
df = pd.read_csv("merged_trips_weather_2022_sample.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'merged_trips_weather_2022_sample.csv'

In [ ]:
# Parse datetime
for col in ["starttime", "start_time", "started_at", "start_timestamp"]:
    if col in df.columns:
        df["start_dt"] = pd.to_datetime(df[col])
        break

df.head()

Intro
Welcome! This interactive analysis explores New York Citi Bike (2022) usage patterns, weather relationships, and station demand hot spots.

Contents:

Trips vs Temperature trends
Most popular stations
Geospatial distribution (map)
Extra insights (hourly × weekday demand)
Recommendations

Dual-Axis Line Chart — Trips vs Temperature

In [ ]:
# Helper functions
def trips_per_day(_df):
    if "start_dt" not in _df.columns:
        return pd.DataFrame(columns=["date","trips"])
    g = _df.groupby(_df["start_dt"].dt.date).size().reset_index(name="trips")
    g["date"] = pd.to_datetime(g["date"])
    return g

def avg_temp_per_day(_df, temp_col):
    if "start_dt" not in _df.columns or temp_col not in _df.columns:
        return pd.DataFrame(columns=["date","avg_temp"])
    g = _df.groupby(_df["start_dt"].dt.date)[temp_col].mean().reset_index(name="avg_temp")
    g["date"] = pd.to_datetime(g["date"])
    return g

# Detect temperature column
temp_cols = [c for c in df.columns if "temp" in c.lower() or "tavg" in c.lower()]
temp_col = temp_cols[0] if temp_cols else None

trips_daily = trips_per_day(df)
temp_daily = avg_temp_per_day(df, temp_col) if temp_col else pd.DataFrame()

fig = go.Figure()
fig.add_trace(go.Scatter(x=trips_daily["date"], y=trips_daily["trips"],
                         mode="lines", name="Trips per day", yaxis="y1"))
if not temp_daily.empty:
    fig.add_trace(go.Scatter(x=temp_daily["date"], y=temp_daily["avg_temp"],
                             mode="lines", name=f"Avg Temp ({temp_col})", yaxis="y2"))
    fig.update_layout(
        xaxis=dict(title="Date"),
        yaxis=dict(title="Trips"),
        yaxis2=dict(title="Avg Temperature", overlaying="y", side="right"),
        hovermode="x unified")
fig.show()

Interpretation:

Peaks in trips align with comfortable temperatures.
Extremely cold or hot days depress ridership.
Seasonality is a major driver of demand.

Most Popular Stations — Bar Chart

In [ ]:
def top_stations(_df, k=20):
    candidates = ["start_station_name","start_station","Start Station Name"]
    col = next((c for c in candidates if c in _df.columns), None)
    if not col:
        return pd.DataFrame(columns=["station","trips"])
    g = _df.groupby(col).size().reset_index(name="trips").sort_values("trips", ascending=False).head(k)
    g.rename(columns={col:"station"}, inplace=True)
    return g

tops = top_stations(df, k=20)
fig = px.bar(tops, x="station", y="trips")
fig.update_layout(xaxis_title="Station", yaxis_title="Trips", xaxis_tickangle=-45)
fig.show()

In [ ]:
Interpretation:

High-throughput stations should receive prioritized replenishment.
Clusters of popular stations point to transit hubs and busy districts

Kepler.gl Map — Start Locations

In [ ]:
# Load trips and weather
trips = pd.read_csv(TRIPS_FP, parse_dates=['started_at', 'ended_at', 'date'], low_memory=False)
weather = pd.read_csv(WEATHER_FP, parse_dates=['date'])

# Normalize station column names if needed
if 'start_station_name' not in trips.columns and 'from_station_name' in trips.columns:
    trips = trips.rename(columns={'from_station_name': 'start_station_name'})
if 'end_station_name' not in trips.columns and 'to_station_name' in trips.columns:
    trips = trips.rename(columns={'to_station_name': 'end_station_name'})

# Ensure coordinate columns exist
required_trip_cols = ['start_station_name', 'end_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng']
missing = [c for c in required_trip_cols if c not in trips.columns]
if missing:
    raise RuntimeError(f'Missing required trip columns: {missing}')

# Add value for aggregation
trips['value'] = 1

# Group by origin-destination and coordinates
df_grouped = (
    trips.groupby([
        'start_station_name', 'end_station_name',
        'start_lat', 'start_lng',
        'end_lat', 'end_lng'
    ])['value']
    .sum()
    .rename('trips')
    .reset_index()
)

print("Original trip count:", len(trips))
print("Aggregated trip sum:", int(df_grouped['trips'].sum()))

# Prepare arcs DataFrame for kepler.gl
arcs_df = pd.DataFrame({
    'start_lat': df_grouped['start_lat'],
    'start_lng': df_grouped['start_lng'],
    'end_lat': df_grouped['end_lat'],
    'end_lng': df_grouped['end_lng'],
    'start_station_name': df_grouped['start_station_name'],
    'end_station_name': df_grouped['end_station_name'],
    'trips': df_grouped['trips'],
})

# limit to top N flows
TOP_N = 500
top_arcs = arcs_df.nlargest(TOP_N, 'trips').copy()

In [ ]:
# 4.Visualize with kepler.gl
# Instantiate kepler.gl map
m = KeplerGl(height=700, data={'popular_trips': top_arcs})

In [ ]:
m

Extra Insights — Hourly × Weekday Heatmap

In [ ]:
if "start_dt" in df.columns:
    df["hour"] = df["start_dt"].dt.hour
    df["weekday"] = df["start_dt"].dt.day_name()
    g = df.groupby(["weekday","hour"]).size().reset_index(name="trips")
    cats = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    g["weekday"] = pd.Categorical(g["weekday"], categories=cats, ordered=True)
    pivot = g.pivot(index="weekday", columns="hour", values="trips").fillna(0)
    fig = px.imshow(pivot, aspect="auto")
    fig.show()

Why this matters:

Commute spikes (morning/evening) show rebalancing time windows.
Weekend peaks suggest moving bikes to leisure areas in advance.

Recommendations

In [ ]:
Based on observed seasonality, station popularity, spatial clusters, and hourly demand:

1.Pre-positioning by Season & Day:

Spring/Fall: increase baseline dock capacity in commuter corridors.
2.Summer weekends: add temporary corrals near parks/waterfronts.
3.Time-boxed Rebalancing Runs:

Run fast loops during commute peaks and weekend late mornings.
4.Micro-zones & Loops:

Cluster nearby stations into micro-zones for efficient servicing.
5.Weather-aware Thresholds:

Raise depletion alerts on temperate days, relax on extreme days.
6.Station-level Targets:

Maintain floor/ceiling targets to prevent shortages and overflows.
7.Measure & Iterate:

Track fill levels and lost trips; re-tune monthly using this analysis.